In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [24]:
url_inicial = 'https://revistas-filologicas.unam.mx/nouatellus/index.php/nt/issue/archive?issuesPage=3#issues'
url_root = 'https://revistas-filologicas.unam.mx/nouatellus/index.php/nt/issue/archive?issuesPage=3#issues'
r=requests.get(url_inicial)

In [25]:
soup = BeautifulSoup(r.text, 'html.parser')

In [26]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', style='float: left; width: 100%;')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://revistas-filologicas.unam.mx/nouatellus/index.php/nt/issue/view/2',
 'https://revistas-filologicas.unam.mx/nouatellus/index.php/nt/issue/view/33',
 'https://revistas-filologicas.unam.mx/nouatellus/index.php/nt/issue/view/1',
 'https://revistas-filologicas.unam.mx/nouatellus/index.php/nt/issue/view/32']

In [27]:
def get_url_items(sopa, url): 
    box = soup.find('div', id='issues')
    volumen=box.findAll('div', style='float: left; width: 100%;')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [28]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://revistas-filologicas.unam.mx/nouatellus/index.php/nt/issue/archive?issuesPage=3#issues


In [29]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

4

In [30]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [31]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Nova Tellus'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Arts and Humanities'
    url='http://www.revistas.unam.mx/index.php/nova_tellus'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('div', class_='tocTitle').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [32]:
list_scraper=list_scraper[0:4]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3


In [33]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Nova Tellus,IV. Humanidades y Ciencias de la Conducta,Arts and Humanities,None,None,http://www.revistas.unam.mx/index.php/nova_tellus,None
1,Nova Tellus,IV. Humanidades y Ciencias de la Conducta,Arts and Humanities,Veinte notas al Fedro,None,http://www.revistas.unam.mx/index.php/nova_tellus,https://revistas-filologicas.unam.mx/nouatellu...
2,Nova Tellus,IV. Humanidades y Ciencias de la Conducta,Arts and Humanities,None,None,http://www.revistas.unam.mx/index.php/nova_tellus,None
3,Nova Tellus,IV. Humanidades y Ciencias de la Conducta,Arts and Humanities,Sabios y sabiduría en Heródoto,None,http://www.revistas.unam.mx/index.php/nova_tellus,https://revistas-filologicas.unam.mx/nouatellu...


In [34]:
df_catalogo.to_csv('Revista193.03.csv', index=False)